### Menu Dataset Preparation

- **Source:** [Kaggle](https://www.kaggle.com/datasets/edoardoscarpaci/italian-food-recipes)

- **Dataset Shape:** 5939 rows × 6 columns
- The dataset is prepared using Mistral 8*7B for the columns "allergens" and "additional information"
- Steps include setting the prompt and converting the dataset to Mistral model input format

- The Dataset is Scalable

- The dataset can be used for Designer (Use Stable diffusion to generate Images)  & Copywriting Agent


### Install necessary libraries

In [ ]:
!pip install --upgrade duckduckgo_search
!pip install --upgrade langchain
!pip install langchain-groq
!pip install wikipedia
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain.agents import Tool
from groq import Groq
import pandas as pd
import ast
import os

In [ ]:
GROQ_API_KEY3=userdata.get('GROQ_API_KEY3')
client = Groq(api_key=GROQ_API_KEY3,)

In [ ]:
llm = ChatGroq(temperature=0.2, #If the temperature is set to a lower value, the model tends to produce more deterministic response
               groq_api_key=GROQ_API_KEY3,
               model_name="mixtral-8x7b-32768",
               max_tokens=20000) #Mistral context window is 32768 tokens, however we are limiting that to 20K tokens.

### Dataset Loading

In [ ]:
df = pd.read_csv("gz_recipe.csv", index_col=0)
df

,Nome,Categoria,Link,Persone/Pezzi,Ingredienti,Steps
0,Tiramisù,Dolci,https://ricette.giallozafferano.it/Tiramisu.html,8,"[['Mascarpone', '750g'], ['Uova', '260g'], ['S...",Per preparare il tiramisù preparate il caffé c...
1,Cookies,Dolci,https://ricette.giallozafferano.it/Cookies.html,12,"[['Farina 00', '195g'], ['Burro', '100g'], ['B...","Per preparare i cookies, assicuratevi che il b..."
2,Pancake allo sciroppo d'acero,Dolci,https://ricette.giallozafferano.it/Pancakes-al...,4,"[['Burro', '25g'], ['Farina 00', '125g'], ['Uo...",Iniziamo la preparazione dei pancake fondendo ...
3,Crema al mascarpone,Dolci,https://ricette.giallozafferano.it/Crema-al-ma...,4,"[['Mascarpone', '500g'], ['Zucchero', '125g'],...",Per preparare la crema al mascarpone versate i...
4,Crepe dolci e salate (ricetta base),Dolci,https://ricette.giallozafferano.it/Crepes-dolc...,15,"[['Uova', '3'], ['Farina 00', '250g'], ['Latte...",Per preparare le crepe dolci e salate iniziate...
...,...,...,...,...,...,...
5934,Bowl di anguria e tofu,Dolci,https://ricette.giallozafferano.it/Bowl-di-ang...,4,"[['Cocomero (anguria)', '450g'], ['Fragole', '...",Per realizzare la bowl di anguria e tofu per p...
5935,Granita bicolore con finto gelato,Dolci,https://ricette.giallozafferano.it/Granita-bic...,4,"[['Cocomero (anguria)', '400g'], ['Melone cant...",Per preparare la granita bicolore con finto ge...
5936,Bicchieri di albicocche e robiola,Dolci,https://ricette.giallozafferano.it/Bicchieri-d...,4,"[['Albicocche', '500g'], ['Zucchero', '50g'], ...",Per preparare i bicchieri di albicocche e robi...
5937,Fiori di Eglefino zucca e pesto,Secondi piatti,https://ricette.giallozafferano.it/Fiori-di-Eg...,2,"[['Eglefino', '2'], ['Zucca mantovana', '1kg']...",Per realizzare i Fiori di Eglefino zucca e pes...


In [ ]:
def get_llm_output(prompt):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="mixtral-8x7b-32768",
    )
    return (chat_completion.choices[0].message.content)


def collect_recipe(df, i):
    """
    collects name and ingredients from the df and rows collects the number of rows upon which operation has to be performed
    df: df containing the name and ingredients "Nome" and "ingredienti"
    rows: number of rows to be collected (default 1)
    """
    name = df["Nome"].iloc[i]
    ingredients = ast.literal_eval(df["Ingredienti"].iloc[i])
    # drop the second value in list
    for i in range(len(ingredients)):
        ingredients[i] = ingredients[i][0]
    wiki_data = wikipedia.run(name)
    return [name, ingredients, wiki_data]


def set_prompt(dish_list):
    name, ingredients, description = dish_list[0], dish_list[1], dish_list[2]
    # Define prompt
    prompt_template = f"""Write a concise summary based on the dish name
    "{name}" and ingredients "{ingredients}". Do not mention the ingredients in final output.
    FINAL CONCISE SUMMARY:"""
    return prompt_template

def set_prompt_allergen(dish_list):
    name, ingredients, description = dish_list[0], dish_list[1], dish_list[2]
    # Define prompt
    prompt_template = f"""Given the name
    "{name}" and ingredients "{ingredients}".
    Find the allergen from the list  [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams]
    OUTPUT should be a list.
    OUTPUT ALLERGEN LIST:<list>"""
    return prompt_template

def set_prompt_option(dish_list):
    name, ingredients, description = dish_list[0], dish_list[1], dish_list[2]
    # Define prompt
    prompt_template = f"""Given the name
    "{name}" and ingredients "{ingredients}".
    Choose values from the list that matches the dish:[Spicy, Vegan, Gluten free, Vegetarian]. reduce the output size in maximum 2 words. return nothing if not found.
    OUTPUT OPTION LIST [Spicy, Vegan, Gluten free, Vegetarian] :"""
    return prompt_template

def find_words_in_sentence(sentence, word_list):
    words_found = []
    for word in word_list:
        if word.lower() in sentence.lower():
            words_found.append(word)
    print("Words found in the sentence:", words_found)
    return words_found


### Using gorQ to extract data for example : Description , Allergens & Additional Information

#### Getting description

In [ ]:
dish_nds_list = collect_recipe(df,4)
print(dish_nds_list)
print("********************************")
basic_prompt = set_prompt(dish_nds_list)
print(basic_prompt)
print("********************************")
get_llm_output(basic_prompt)

['Crepe dolci e salate (ricetta base)', ['Uova', 'Farina 00', 'Latte intero', 'Burro'], 'No good Wikipedia Search Result was found']
********************************
Write a concise summary based on the dish name
    "Crepe dolci e salate (ricetta base)" and ingredients "['Uova', 'Farina 00', 'Latte intero', 'Burro']". Do not mention the ingredients in final output.
    FINAL CONCISE SUMMARY:
********************************


'"Crepe Dolci e Salate (Base Recipe)" refers to a versatile base recipe for creating both sweet and savory crepes. This Italian dish, known as "Crepe Dolci e Salate," is a culinary staple that can be customized to suit various taste preferences. The recipe utilizes four essential ingredients: eggs, fine flour (00), whole milk, and butter, which come together to create the crepe\'s thin, delicate, and flexible texture. Once mastered, the base recipe allows for an endless array of fillings and toppings, making it a delightful and adaptable dish for any occasion.'

#### Allergen list

In [ ]:

basic_prompt = set_prompt_allergen(dish_nds_list)
print(basic_prompt)
print("********************************")
output = get_llm_output(basic_prompt)
print(output)

Given the name
    "Cookies" and ingredients "['Farina 00', 'Burro', 'Bicarbonato', 'Uova', 'Zucchero di canna', 'Zucchero', 'Gocce di cioccolato fondente', 'Sale fino']". 
    Find the allergen from the list  [Cereals, Crustaceans, Egg, Fish, Peanuts, SOYBEAN, Latte, Nuts, Celery, Mustard, Sesame seeds, Sulfur dioxide and sulphites, Shell, Clams]
    OUTPUT should be a list.
    OUTPUT ALLERGEN LIST:<list>
********************************
To identify potential allergens in the given ingredients, we need to compare the ingredients with the list of common allergens. Here is the breakdown:

1. 'Farina 00' is a type of flour made from wheat, which falls under 'Cereals'.
2. 'Burro' is butter, and it contains 'Latte' (milk).
3. 'Bicarbonato' is baking soda and does not contain any common allergens.
4. 'Uova' is eggs, which are an 'Egg' allergen.
5. 'Zucchero di canna' and 'Zucchero' are both types of sugar and do not contain any common allergens.
6. 'Gocce di cioccolato fondente' are dark c

In [ ]:
allergen_list = ["Cereals", "Crustaceans", "Egg", "Fish", "Peanuts", "SOYBEAN", "Latte", "Nuts", "Celery", "Mustard", "Sesame seeds", "Sulfur dioxide and sulphites", "Shell", "Clams"]
find_words_in_sentence(output, allergen_list)

Words found in the sentence: ['Cereals', 'Egg', 'SOYBEAN', 'Latte']


['Cereals', 'Egg', 'SOYBEAN', 'Latte']

### Additional Information

In [ ]:

basic_prompt = set_prompt_option(dish_nds_list)
print(basic_prompt)
print("********************************")
output = get_llm_output(basic_prompt)
print(output)
print("********************************")

dish_option_list = ["Spicy", "Vegan", "Gluten free", "Vegetarian"]
find_words_in_sentence(output, dish_option_list)

Given the name
    "Crepe dolci e salate (ricetta base)" and ingredients "['Uova', 'Farina 00', 'Latte intero', 'Burro']".
    Choose values from the list that matches the dish:[Spicy, Vegan, Gluten free, Vegetarian]. reduce the output size in maximum 2 words. return nothing if not found.
    OUTPUT OPTION LIST [Spicy, Vegan, Gluten free, Vegetarian] :
********************************
Vegetarian
********************************
Words found in the sentence: ['Vegetarian']


['Vegetarian']

### dataset preparation

In [ ]:

for i in range(df.shape[0]):
    dish_data = {}

    # name and description
    dish_nds_list = collect_recipe(df,i)
    basic_prompt = set_prompt(dish_nds_list)
    dish_data["name"] = dish_nds_list[0]
    dish_data["ingredients"] = dish_nds_list[1]
    dish_data["description"] = get_llm_output(basic_prompt)

    # allegen list
    basic_prompt = set_prompt_allergen(dish_nds_list)
    output = get_llm_output(basic_prompt)
    allergen_list = ["Cereals", "Crustaceans", "Egg", "Fish", "Peanuts", "SOYBEAN", "Latte", "Nuts", "Celery", "Mustard", "Sesame seeds", "Sulfur dioxide and sulphites", "Shell", "Clams"]
    dish_data["allergen"] = find_words_in_sentence(output, allergen_list)

    # options for dish
    basic_prompt = set_prompt_option(dish_nds_list)
    output = get_llm_output(basic_prompt)
    dish_option_list = ["Spicy", "Vegan", "Gluten free", "Vegetarian"]
    dish_data["option"] = find_words_in_sentence(output, dish_option_list)

    print(dish_data)
    break


/Users/wenda/anaconda3/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/wenda/anaconda3/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Words found in the sentence: ['Egg']
Words found in the sentence: ['Gluten free', 'Vegetarian']
{'name': 'Tiramisù', 'ingredients': ['Mascarpone', 'Uova', 'Savoiardi', 'Zucchero', 'Caffè', 'Cacao amaro in polvere'], 'description': 'Tiramisù is a classic Italian dish made with rich and creamy mascarpone cheese, beautifully combined with sweetened eggs and delicate savoiardi biscuits, all soaked in a delightful coffee infusion, and finished with a dusting of bitter cocoa powder.', 'allergen': ['Egg'], 'option': ['Gluten free', 'Vegetarian']}


In [ ]:
# creating an empty DataFrame
df_new = pd.DataFrame(columns=["name", "ingredients", "description", "allergen", "option"])

for i in range(df.shape[0]):
    dish_data = {}

    # name and description
    dish_nds_list = collect_recipe(df,i)
    basic_prompt = set_prompt(dish_nds_list)
    dish_data["name"] = dish_nds_list[0]
    dish_data["ingredients"] = dish_nds_list[1]
    dish_data["description"] = get_llm_output(basic_prompt)

    # allergen list
    basic_prompt = set_prompt_allergen(dish_nds_list)
    output = get_llm_output(basic_prompt)
    allergen_list = ["Cereals", "Crustaceans", "Egg", "Fish", "Peanuts", "SOYBEAN", "Latte", "Nuts", "Celery", "Mustard", "Sesame seeds", "Sulfur dioxide and sulphites", "Shell", "Clams"]
    dish_data["allergen"] = find_words_in_sentence(output, allergen_list)

    # options for dish
    basic_prompt = set_prompt_option(dish_nds_list)
    output = get_llm_output(basic_prompt)
    dish_option_list = ["Spicy", "Vegan", "Gluten free", "Vegetarian"]
    dish_data["option"] = find_words_in_sentence(output, dish_option_list)

    # appending the dictionary as a new row to the DataFrame
    df_new = df_new.append(dish_data, ignore_index=True)

# show the DataFrame
df_new


In [ ]:
df_new

,name,ingredients,description,allergen,option
0,Tiramisù,"[Mascarpone, Uova, Savoiardi, Zucchero, Caffè,...",Tiramisù is a classic Italian dessert made wit...,[Egg],"[Gluten free, Vegetarian]"


### Conversion of dataset to Open LLM prompt

* I have prepared dataset for 1517


In [ ]:
import pandas as pd
df=pd.read_csv("/content/menuDataset.csv")
# Assuming your DataFrame is named 'df'
df = df.drop(df.columns[0], axis=1)

In [ ]:
df.head(20)

,name,ingredients,description,allergen,option
0,Tiramisù,"['Mascarpone', 'Uova', 'Savoiardi', 'Zucchero'...","Tiramisù is a delightful Italian dessert, ofte...",['Egg'],"['Gluten free', 'Vegetarian']"
1,Cookies,"['Farina 00', 'Burro', 'Bicarbonato', 'Uova', ...","These ""Cookies"" are a delightful treat, classi...","['Cereals', 'Crustaceans', 'Egg', 'Fish', 'Pea...",[]
2,Pancake allo sciroppo d'acero,"['Burro', 'Farina 00', 'Uova', 'Latte intero',...","The dish ""Pancake allo sciroppo d'acero"" featu...","['Egg', 'Latte']",['Vegetarian']
3,Crema al mascarpone,"['Mascarpone', 'Zucchero', 'Tuorli', 'Acqua', ...","""Crema al mascarpone"" is a delightful and crea...","['Egg', 'Latte', 'Nuts']","['Gluten free', 'Vegetarian']"
4,Crepe dolci e salate (ricetta base),"['Uova', 'Farina 00', 'Latte intero', 'Burro']","""Crepe Dolci e Salate (Basic Recipe)"" refers t...","['Cereals', 'Egg', 'Latte']",['Vegetarian']
5,Lasagne alla Bolognese,"['Semola di grano duro rimacinata', 'Farina 00...","""Lasagne alla Bolognese"" is a traditional Ital...","['Cereals', 'Crustaceans', 'Egg', 'Fish', 'Pea...",[]
6,Besciamella,"['Latte intero', 'Burro', 'Farina 00', 'Sale f...","Besciamella is a classic Italian sauce, also k...","['Cereals', 'Latte']","['Gluten free', 'Vegetarian']"
7,Pasta per la pizza,"['Farina Manitoba', 'Farina 00', 'Acqua', 'Sal...","""Pasta per la pizza"" is a type of dough made u...",['Cereals'],['Vegetarian']
8,Biscotti di Pan di zenzero (gingerbread),"['Zenzero in polvere', 'Farina 00', 'Chiodi di...","""Biscotti di Pan di zenzero"" are traditional I...","['Cereals', 'Crustaceans', 'Egg']","['Spicy', 'Vegan', 'Gluten free', 'Vegetarian']"
9,Spaghetti alla Carbonara,"['Spaghetti', 'Guanciale', 'Tuorli', 'Pecorino...","""Spaghetti alla Carbonara"" is a traditional It...","['Cereals', 'Egg']",[]


In [ ]:
# Create the 'Text' column
df['Text'] = df.apply(lambda row: {
    "ingredients": row['ingredients'],
    "description": row['description'],
    "allergen": row['allergen'],
    "Additional_information": row['option']
}, axis=1)

df.head(20)

,name,ingredients,description,allergen,option,Text
0,Tiramisù,"['Mascarpone', 'Uova', 'Savoiardi', 'Zucchero'...","Tiramisù is a delightful Italian dessert, ofte...",['Egg'],"['Gluten free', 'Vegetarian']","{'ingredients': '['Mascarpone', 'Uova', 'Savoi..."
1,Cookies,"['Farina 00', 'Burro', 'Bicarbonato', 'Uova', ...","These ""Cookies"" are a delightful treat, classi...","['Cereals', 'Crustaceans', 'Egg', 'Fish', 'Pea...",[],"{'ingredients': '['Farina 00', 'Burro', 'Bicar..."
2,Pancake allo sciroppo d'acero,"['Burro', 'Farina 00', 'Uova', 'Latte intero',...","The dish ""Pancake allo sciroppo d'acero"" featu...","['Egg', 'Latte']",['Vegetarian'],"{'ingredients': '['Burro', 'Farina 00', 'Uova'..."
3,Crema al mascarpone,"['Mascarpone', 'Zucchero', 'Tuorli', 'Acqua', ...","""Crema al mascarpone"" is a delightful and crea...","['Egg', 'Latte', 'Nuts']","['Gluten free', 'Vegetarian']","{'ingredients': '['Mascarpone', 'Zucchero', 'T..."
4,Crepe dolci e salate (ricetta base),"['Uova', 'Farina 00', 'Latte intero', 'Burro']","""Crepe Dolci e Salate (Basic Recipe)"" refers t...","['Cereals', 'Egg', 'Latte']",['Vegetarian'],"{'ingredients': '['Uova', 'Farina 00', 'Latte ..."
5,Lasagne alla Bolognese,"['Semola di grano duro rimacinata', 'Farina 00...","""Lasagne alla Bolognese"" is a traditional Ital...","['Cereals', 'Crustaceans', 'Egg', 'Fish', 'Pea...",[],{'ingredients': '['Semola di grano duro rimaci...
6,Besciamella,"['Latte intero', 'Burro', 'Farina 00', 'Sale f...","Besciamella is a classic Italian sauce, also k...","['Cereals', 'Latte']","['Gluten free', 'Vegetarian']","{'ingredients': '['Latte intero', 'Burro', 'Fa..."
7,Pasta per la pizza,"['Farina Manitoba', 'Farina 00', 'Acqua', 'Sal...","""Pasta per la pizza"" is a type of dough made u...",['Cereals'],['Vegetarian'],"{'ingredients': '['Farina Manitoba', 'Farina 0..."
8,Biscotti di Pan di zenzero (gingerbread),"['Zenzero in polvere', 'Farina 00', 'Chiodi di...","""Biscotti di Pan di zenzero"" are traditional I...","['Cereals', 'Crustaceans', 'Egg']","['Spicy', 'Vegan', 'Gluten free', 'Vegetarian']","{'ingredients': '['Zenzero in polvere', 'Farin..."
9,Spaghetti alla Carbonara,"['Spaghetti', 'Guanciale', 'Tuorli', 'Pecorino...","""Spaghetti alla Carbonara"" is a traditional It...","['Cereals', 'Egg']",[],"{'ingredients': '['Spaghetti', 'Guanciale', 'T..."


In [ ]:
df_transform=df.copy()

### Prepare a prompt for the Model

#### Prompt Format

**Input:** Tiramisu

**Output:**
```json
{
  "ingredients": ["Mascarpone", "Uova", "Savoiardi", "Zucchero", "Caffè", "Cacao amaro in polvere"],
  "description": "Tiramisù is a delightful Italian dessert, often presented in a layered format, featuring the use of rich and creamy mascarpone cheese, blended with eggs to create a smooth and decadent cream. This luscious cream is then combined with a delicate balance of sugar and bold, aromatic coffee. The mixture is then artfully layered with airy savoiardi biscuits, which serve to soak up the coffee flavors while adding a satisfying crunch. The dish is finally finished off with a dusting of bitter cocoa powder, adding an elegant touch that complements the sweet and creamy elements of the dessert.",
  "allergen": ["Egg"],
  "Additional_information": ["Gluten free", "Vegetarian"]
}


In [ ]:
INS = """<s>[INST]As a restaurant menu manager, your role is to gather below informations based on input  (Name of the dish).
    generate the output"""

In [ ]:
for i in range(len(df_transform)):
  input = df_transform['name'][i]
  output = df_transform['Text'][i]
  prompt = f"""{INS} ### Input data:{input} ### Output:{output}"""
  df_transform['Text'].iloc[i] = prompt
df_transform.drop(columns=['name','ingredients','description','allergen','option'], inplace=True)
df_transform.head()

,Text
0,"<s>[INST]As a restaurant menu manager, your ro..."
1,"<s>[INST]As a restaurant menu manager, your ro..."
2,"<s>[INST]As a restaurant menu manager, your ro..."
3,"<s>[INST]As a restaurant menu manager, your ro..."
4,"<s>[INST]As a restaurant menu manager, your ro..."


In [ ]:
df_transform.shape

(1517, 1)

In [ ]:
df_transform.head(20)

,Text
0,"<s>[INST]As a restaurant menu manager, your ro..."
1,"<s>[INST]As a restaurant menu manager, your ro..."
2,"<s>[INST]As a restaurant menu manager, your ro..."
3,"<s>[INST]As a restaurant menu manager, your ro..."
4,"<s>[INST]As a restaurant menu manager, your ro..."
5,"<s>[INST]As a restaurant menu manager, your ro..."
6,"<s>[INST]As a restaurant menu manager, your ro..."
7,"<s>[INST]As a restaurant menu manager, your ro..."
8,"<s>[INST]As a restaurant menu manager, your ro..."
9,"<s>[INST]As a restaurant menu manager, your ro..."


In [ ]:
import pandas as pd

# Set the display options to show more rows and columns
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

# Display the first 20 rows of the DataFrame
df_transform.head(20)

,Text
0,"<s>[INST]As a restaurant menu manager, your role is to gather below informations based on input (Name of the dish).\n generate the output ### Input data:Tiramisù ### Output:{'ingredients': ""['Mascarpone', 'Uova', 'Savoiardi', 'Zucchero', 'Caffè', 'Cacao amaro in polvere']"", 'description': 'Tiramisù is a delightful Italian dessert, often presented in a layered format, featuring the use of rich and creamy mascarpone cheese, blended with eggs to create a smooth and decadent cream. This luscious cream is then combined with a delicate balance of sugar and bold, aromatic coffee. The mixture is then artfully layered with airy savoiardi biscuits, which serve to soak up the coffee flavors while adding a satisfying crunch. The dish is finally finished off with a dusting of bitter cocoa powder, adding an elegant touch that complements the sweet and creamy elements of the dessert.', 'allergen': ""['Egg']"", 'Additional_information': ""['Gluten free', 'Vegetarian']""}"
1,"<s>[INST]As a restaurant menu manager, your role is to gather below informations based on input (Name of the dish).\n generate the output ### Input data:Cookies ### Output:{'ingredients': ""['Farina 00', 'Burro', 'Bicarbonato', 'Uova', 'Zucchero di canna', 'Zucchero', 'Gocce di cioccolato fondente', 'Sale fino']"", 'description': 'These ""Cookies"" are a delightful treat, classically made with fine ""Farina 00,"" rich ""Burro,"" leavening ""Bicarbonato,"" ""Uova"" (eggs), and a blend of ""Zucchero di canna"" and ""Zucchero"" (sugars) to create a perfectly sweet foundation. Indulgent ""Gocce di cioccolato fondente"" (chocolate chips) are generously mixed in for added luxury, while a pinch of ""Sale fino"" (fine salt) heightens all the flavors.', 'allergen': ""['Cereals', 'Crustaceans', 'Egg', 'Fish', 'Peanuts', 'SOYBEAN', 'Latte', 'Nuts', 'Celery', 'Mustard', 'Sesame seeds', 'Sulfur dioxide and sulphites', 'Shell', 'Clams']"", 'Additional_information': '[]'}"
2,"<s>[INST]As a restaurant menu manager, your role is to gather below informations based on input (Name of the dish).\n generate the output ### Input data:Pancake allo sciroppo d'acero ### Output:{'ingredients': ""['Burro', 'Farina 00', 'Uova', 'Latte intero', 'Lievito in polvere per dolci', 'Zucchero', 'Sciroppo di acero']"", 'description': 'The dish ""Pancake allo sciroppo d\'acero"" features classic pancakes drizzled with sweet, rich maple syrup. The pancakes are made from a combination of finely milled ""00"" flour, eggs, whole milk, and leavened with powdered sweet yeast, with a touch of sugar added for a hint of sweetness. The star ingredient is the maple syrup, elevating the pancakes with its distinct, robust flavor.', 'allergen': ""['Egg', 'Latte']"", 'Additional_information': ""['Vegetarian']""}"
3,"<s>[INST]As a restaurant menu manager, your role is to gather below informations based on input (Name of the dish).\n generate the output ### Input data:Crema al mascarpone ### Output:{'ingredients': ""['Mascarpone', 'Zucchero', 'Tuorli', 'Acqua', 'Cacao amaro in polvere']"", 'description': '""Crema al mascarpone"" is a delightful and creamy dessert, featuring a rich combination of mascarpone cheese and sugar, sweetened with a hint of powdered bitter cocoa.', 'allergen': ""['Egg', 'Latte', 'Nuts']"", 'Additional_information': ""['Gluten free', 'Vegetarian']""}"
4,"<s>[INST]As a restaurant menu manager, your role is to gather below informations based on input (Name of the dish).\n generate the output ### Input data:Crepe dolci e salate (ricetta base) ### Output:{'ingredients': ""['Uova', 'Farina 00', 'Latte intero', 'Burro']"", 'description': '""Crepe Dolci e Salate (Basic Recipe)"" refers to a versatile dish that can be made sweet or savory, using a base recipe involving eggs, fine flour (00), whole milk, and butter. The recipe provides a foundation for creating various delicious crepe recipes, to be enjoyed either as a dessert, a main course, or a snack, limited only by one\'s imagination and taste pr

In [ ]:
df_transform.to_csv('MenuDataset.csv')